# Pantograph Example

The only interface for interacting with Pantograph is the `Server` class. It can be used either standalone (with no Lean project specified) or in a Lean project in order to access the project's symbols (e.g. Mathlib).

The server's `imports` argument must be specified as a list of Lean modules to import. With no import statements, there are no symbols available and no useful work can be done. By default, `imports` is `["Init"]`.

In [1]:
from pantograph import Server

server = Server()

We can initialize a proof by providing the target statement.

In [2]:
state0 = server.goal_start("forall (p : Prop), p -> p")

This invocation creates a *goal state*, which consists of a finite number of goals. 

In [3]:
state0

GoalState(state_id=0, goals=[Goal(variables=[], target='forall (p : Prop), p -> p', name=None, is_conversion=False)], _sentinel=[])

Execute tactics on the goal state via `Server.goal_tactic`:

In [4]:
state1 = server.goal_tactic(state0, goal_id=0, tactic="intro")
state1

GoalState(state_id=1, goals=[Goal(variables=[Variable(t='Prop', v=None, name='p✝')], target='p✝ → p✝', name=None, is_conversion=False)], _sentinel=[])

Recover the usual string form of a goal by the `str` function:

In [5]:
str(state1.goals[0])

'p✝ : Prop\n⊢ p✝ → p✝'

When a tactic fails, it throws an exception:

In [6]:
try:
    state2 = server.goal_tactic(state1, goal_id=0, tactic="assumption")
    print("Should not reach this")
except Exception as e:
    print(e)

["tactic 'assumption' failed\np✝ : Prop\n⊢ p✝ → p✝"]


A state with no goals is considered solved

In [7]:
state2 = server.goal_tactic(state1, goal_id=0, tactic="intro h")
state3 = server.goal_tactic(state2, goal_id=0, tactic="exact h")
state3

GoalState(state_id=3, goals=[], _sentinel=[])

Execute `Server.gc()` to clean up unused goals once in a while

In [8]:
server.gc()

## Loading Projects

Pantograph would not be useful if it could not load symbols from other projects. In `examples/Example` is a standard Lean 4 project, with its toolchain version precisely equal to the toolchain version of Pantograph. Executing `lake new PROJECT_NAME` or `lake init` in an empty folder initializes a project according to this specification. To use a project in Pantograph, compile the project by running `lake build` in its root directory. This sets up output folders and builds the binary Lean files.

Load the example project by providing `project_path` and `lean_path` to `Server`:

In [9]:
import subprocess, os
from pathlib import Path
project_path = Path(os.getcwd()).resolve() / 'Example'
print(f"$PWD: {project_path}")
server = Server(imports=['Example'], project_path=project_path)

$PWD: /home/aniva/Projects/atp/PyPantograph/examples/Example
$LEAN_PATH: b'././.lake/packages/batteries/.lake/build/lib:././.lake/packages/aesop/.lake/build/lib:././.lake/build/lib:/home/aniva/.elan/toolchains/leanprover--lean4---v4.12/lib/lean\n'


With the project loaded, all dependencies of the project, be it Mathlib or Aesop, are now available.

In [10]:
state0 = server.goal_start("forall (p q: Prop), Or p q -> Or q p")
state1 = server.goal_tactic(state0, goal_id=0, tactic="aesop")
state1.is_solved

True

## Reading Symbols

Pantograph can also query proof states from a project by directly calling into Lean's compiler internals. Run the Lean compiler on a Lean file via `Server.tactic_invocations`. Feeding in the absolute path is preferred.

In [11]:
units, invocations = server.tactic_invocations(project_path / "Example.lean")
for i, u in enumerate(units):
    print(f"==== #{i} ====")
    print(u)
print("==== Invocations ====")
for i in invocations:
    print(f"{i.before}\n{i.tactic}\n{i.after}\n")

==== #0 ====
/-- Ensure that Aesop is running -/
example : α → α :=
  by aesop


==== #1 ====
example : ∀ (p q: Prop), p ∨ q → q ∨ p := by
  intro p q h
  -- Here are some comments
  cases h
  . apply Or.inr
    assumption
  . apply Or.inl
    assumption

==== Invocations ====
α : Sort ?u.7
⊢ α → α
aesop


⊢ ∀ (p q : Prop), p ∨ q → q ∨ p
intro p q h
p q : Prop
h : p ∨ q
⊢ q ∨ p

p q : Prop
h : p ∨ q
⊢ q ∨ p
cases h
case inl
p q : Prop
h✝ : p
⊢ q ∨ p
case inr p q : Prop h✝ : q ⊢ q ∨ p

case inl
p q : Prop
h✝ : p
⊢ q ∨ p
apply Or.inr
case inl.h
p q : Prop
h✝ : p
⊢ p

case inl.h
p q : Prop
h✝ : p
⊢ p
assumption


case inr
p q : Prop
h✝ : q
⊢ q ∨ p
apply Or.inl
case inr.h
p q : Prop
h✝ : q
⊢ q

case inr.h
p q : Prop
h✝ : q
⊢ q
assumption




### Loading a theorem for the agent to prove

For this to work, write `sorry` in the place where you would like your agent to do work, for example
```lean
example : (p q: Prop): Or p q -> Or q p := sorry
```
Then use the `Server.load_sorry`. As long as you only have one statement in `command`, it will give you exactly one goal state. A command with no `sorry`s results in no goal states.

Note: Since a syntactically incorrect command will not generate `sorry`s, they will be sliently ignored by the frontend. Check if this is the case if the function returns no goals.

In [12]:
state0, = server.load_sorry("example (p q: Prop): Or p q -> Or q p := sorry")
print(state0)

GoalState(state_id=2, goals=[Goal(variables=[Variable(t='Prop', v=None, name='p'), Variable(t='Prop', v=None, name='q')], target='p ∨ q → q ∨ p', name=None, is_conversion=False)], _sentinel=[0])
